Munges CSV to JSON for playlist graph.

In [21]:
import json
import random

# analytics api wranglers
import lyricsgenius # has aggressive versioning requirements
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [76]:
rows = []
with open('thumbprint.csv') as f:
    cols = [s.strip() for s in f.readline().split(',')]
    for line in f:
        if '---' in line:
            rows.append( {} )
        else:
            # linker will remove duplicates
            rows.append( dict(zip(cols, [s.strip() for s in line.split(',')])) )

        # else: rows.append( dict(empty=True) )
rows[-1]

{'artist': 'kid architect', 'song': 'terror tone'}

In [114]:
links = []

u = rows[0]
nodes = [] if u == {} else [u]

i,j = -1,-1 # indexes of u,v
k = 0 # duplicates count

for v in rows[1:]:
    if v == {}:
        j = -1 # separators don't generate links
    elif v in nodes:
        j = nodes.index(v) # update target pointer
        k += 1
    else:
        j = len(nodes) # push & update target
        nodes.append(v)
    
    # if we did generate a link, add it
    if i != -1 and j != -1:
        links.append( dict(source=i, target=j) )
        
    # target becomes source
    i = j
    u = v
    
k

39

In [115]:
for i,u in enumerate(nodes):
    for j in range(i+1, len(nodes)):
        v = nodes[j]
        if u['artist'] == v['artist']:
            links.append( dict(source=i, target=j, fake=True))

In [116]:
links

[{'source': 0, 'target': 1},
 {'source': 1, 'target': 2},
 {'source': 2, 'target': 3},
 {'source': 3, 'target': 4},
 {'source': 4, 'target': 5},
 {'source': 5, 'target': 6},
 {'source': 6, 'target': 7},
 {'source': 7, 'target': 8},
 {'source': 8, 'target': 9},
 {'source': 9, 'target': 10},
 {'source': 10, 'target': 11},
 {'source': 11, 'target': 12},
 {'source': 12, 'target': 13},
 {'source': 13, 'target': 14},
 {'source': 14, 'target': 15},
 {'source': 15, 'target': 16},
 {'source': 16, 'target': 17},
 {'source': 17, 'target': 18},
 {'source': 18, 'target': 19},
 {'source': 19, 'target': 20},
 {'source': 20, 'target': 21},
 {'source': 21, 'target': 22},
 {'source': 22, 'target': 23},
 {'source': 23, 'target': 24},
 {'source': 24, 'target': 25},
 {'source': 25, 'target': 26},
 {'source': 26, 'target': 27},
 {'source': 27, 'target': 28},
 {'source': 28, 'target': 29},
 {'source': 29, 'target': 30},
 {'source': 30, 'target': 31},
 {'source': 31, 'target': 32},
 {'source': 32, 'target': 3

#### Generate lyrics dictionary.

In [5]:
with open('config/token.txt', 'r') as f:
    genius_token = f.readline().strip()
    
lg = lyricsgenius.Genius(genius_token)
man = SpotifyClientCredentials(client_id=spotify_id,
                               client_secret=spotify_token)

In [6]:
folio = []
with open('gen/lyrics-dict.json') as f:
    folio = json.load(f)

In [7]:
[[u['song'], u['lyrics'][0:3]] for u in folio if 'word' in u['song']]

[["don't say a word",
  ['I am your poison candygram',
   "The love that's meant to fade away",
   "Vade retro, alter ego, move aside, I'm choking on this life"]],
 ['words as weapons',
  ['All I really want is something beautiful to say',
   'All I really want is something beautiful to say',
   'Keep me locked up in your broken mind']],
 ['famous last words',
  ['Now I know', "That I can't make you stay", "But where's your heart?"]]]

In [9]:
# supplement file if incomplete
def fetchLyrics(filename, ret):
    key, lyrics = None, None
    
    with open(filename) as f:
        for line in f:
            if (line[0] is '#'):
                if key:
                    # push collected lyrics
                    song, artist = key.split(', ')
                    idx = [i for i,u in enumerate(nodes) if song in u['song'] \
                                                       and artist in u['artist']]
                        # ASSERT: checking artist too yields unambiguity
                        
                    # ...unless they're already present (VERIFY)
                    if len(idx) > 0 and not folio.any(u => u.id == idx):
                        folio.append({'artist': artist, 'song': song,
                                      'lyrics': lyrics, 'id': idx[0]})
                    
                key = line[1:-1].strip() # song, artist
                lyrics = []
            elif key:
                s = line.strip()
                if '[' not in s and s is not '':
                    lyrics.append(s)

fetchLyrics('config/transcriptions.txt', folio)
fetchLyrics('config/supplements.txt', folio)

# scrape Genius for everything else
# FIXME: not updated for folio-as-list.
for u in nodes:
    id = u['song'] + ', ' + u['artist']
    
    if len([v for v in folio if u['song'] in v['song'] and u['artist'] in v['artist']]) == 0:
        try:
            res = lg.search_song(u['song'], u['artist'])
            
            verses = [s for s in res.lyrics.split('\n') \
                      if '[' not in s and s is not '']
            folio[id] = verses
        except:
            pass

In [12]:
folio.sort(key = lambda u: u['id'])

In [14]:
# json.dumps(folio)
with open('gen/lyrics-dict.json', 'w') as f:
    f.write(json.dumps(folio))

#### Annotate tracks with duration.

In [29]:
with open('config/token.txt', 'r') as f:
    spotify_id = f.readline().strip()
    spotify_token = f.readline().strip()

man = spotipy.oauth2.SpotifyClientCredentials(
        client_id=spotify_id, client_secret=spotify_token)
sp = spotipy.Spotify(client_credentials_manager=man)

In [33]:
# debug shim
import asyncio

res = sp.search(q='track:' + u['song'] + ' artist:' + u['artist'],
                type='track', limit=1)
if res:
    v = res['tracks']['items']
    return v[0].keys()

SpotifyOauthError: Bad Request

In [ ]:
for i,u in enumerate(nodes):
    res = sp.search(q='track:' + u['song'] + ' artist:' + u['artist'],
                        type='track', limit=1)

    v = res['tracks']['items']
    if len(v) > 0:
        u['duration'] = v[0]['duration_ms'] / 1000
        u['year'] = v[0]['album']['release_date'].split('-')[0]
        u['weight'] = v[0]['popularity']
        
    if i%50 == 0:
        print('...')

    # seconds = v['duration_ms'] / 1000
    # m,s = int(seconds//60), int(seconds%60)

Workaround while Spotipy is broken - use existing scrape.

In [117]:
with open('gen/thumbprint-graph-song.json') as f:
    legacy = json.load(f)['nodes']

In [118]:
for i,u in enumerate(nodes):
    res = [v for v in legacy if v['artist'] == u['artist'] and v['song'] == u['song']]
    if res != []:
        nodes[i] = res[0]

In [119]:
for i,u in enumerate(nodes):
    u['id'] = i

with open("gen/thumbprint-graph.json", "w") as f:
    f.write( json.dumps(dict(nodes=nodes, links=links)) )

#### Associational lyrics reads

In [79]:
g = links[random.randint(0, len(links))]
# for g in links:
i,j = g['source'], g['target']
print(nodes[i], nodes[j])

lhs = random.choice(folio[i]['lyrics'])
rhs = random.choice(folio[j]['lyrics'])
lhs, rhs

{'song': 'shy', 'artist': 'sonata arctica'} {'song': 'ghosting', 'artist': 'daylight dies'}


NameError: name 'folio' is not defined

~~FIXME: Cache lyrics? Project onto the actual links, not UI space. Vary transmissions in realtime.~~

FIXME: Spotipy client credentials flow broke, but I want key information.  
Scrape Tunebat? (Also, explore Tunebat.)  
Switch to ExpressJS wrapper?? https://spotify-client-credentials.glitch.me/

#### Visualization (defunct)

Sadly, the 'a-scene' likes to take over the entire notebook, instead of being a widget. 